Task 1. K-means (5 points)

The following data points are given in Euclidean space:

| Point | X    | Y    |
|-------|------|------|
| A     | 1.2  | 0.8  |
| B     | -0.6 | -1.3 |
| C     | -0.8 | 0.2  |
| D     | 0.2  | 0.3  |

Point A and C are initially assigned to cluster centroid C1, points B and D are initially assigned to cluster centroid C2.

In [264]:
import numpy as np

In [265]:
data = np.array([[1.2, .8],
                 [-.6, -1.3],
                 [-.8, .2],
                 [.2, .3]])

data

array([[ 1.2,  0.8],
       [-0.6, -1.3],
       [-0.8,  0.2],
       [ 0.2,  0.3]])

In [266]:
# Initial conditions
initial_assigned_labels = np.array(['C1', 'C2', 'C1', 'C2'])

In [267]:
def calc_centroids(data, assigned_labels):
    labels = set(assigned_labels)

    centroids = dict()
    for label in labels:
        cluster = data[assigned_labels == label]
        centroids[label] = np.mean(cluster, axis=0)

    return centroids

In [268]:
initial_centroids = calc_centroids(data, initial_assigned_labels)

initial_centroids

{'C1': array([0.2, 0.5]), 'C2': array([-0.2, -0.5])}

In [269]:
def d(p1, p2): return np.linalg.norm(p1 - p2)

In [270]:
def kmeans_step(data, assigned_labels):
    assigned_labels = assigned_labels.copy()
    centroids = calc_centroids(data, assigned_labels)

    for i, point in enumerate(data):
        closest = assigned_labels[i]
        closest_d = d(point, centroids[closest])

        for centroid_label, centroid_point in centroids.items():
            dist = d(point, centroid_point)
            if dist < closest_d: closest, closest_d = centroid_label, dist

        assigned_labels[i] = closest

    return assigned_labels

In [271]:
def kmeans(data, initial_assigned_labels):
    prev_assigned_labels = kmeans_step(data, initial_assigned_labels)

    while True:
        assigned_labels = kmeans_step(data, prev_assigned_labels)

        # should be fine, since only 4 points (usually we go by centroids and not equality)
        if np.array_equal(assigned_labels, prev_assigned_labels): break
        else: prev_assigned_labels = assigned_labels

    return assigned_labels

In [272]:
kmeans_labels = kmeans(data, initial_assigned_labels)

kmeans_labels

array(['C1', 'C2', 'C2', 'C1'], dtype='<U2')

In [273]:
def silhouette(data, assigned_labels):
    """only for k = 2"""
    _silhouette = []

    for point, label in zip(data, assigned_labels):
        cum_dist_in_cluster, point_in_cluster_count = 0, -1
        # since we count the point itself as well
        cum_dist_out_cluster, point_out_cluster_count = 0, 0
        for other_point, other_label in zip(data, assigned_labels):
            if label == other_label:
                cum_dist_in_cluster += d(point, other_point)
                point_in_cluster_count += 1
            else:
                cum_dist_out_cluster += d(point, other_point)
                point_out_cluster_count += 1

        avg_dist_out_cluster = cum_dist_out_cluster / point_out_cluster_count if point_out_cluster_count > 0 else 0
        avg_dist_in_cluster = cum_dist_in_cluster / point_in_cluster_count if point_in_cluster_count > 0 else 0
        sil = (avg_dist_out_cluster - avg_dist_in_cluster) / max(avg_dist_out_cluster, avg_dist_in_cluster)

        _silhouette.append(sil)

    return _silhouette

In [274]:
silhouette(data, initial_assigned_labels), silhouette(data, kmeans_labels)

([-0.06997526302916779,
  0.163918335734697,
  -0.39698557819998675,
  -0.4065974368219471],
 [0.5393278339042583,
  0.3355133343754012,
  0.021499716537346912,
  0.19964406640384613])